##### Tool creating and binding using `@tool` decorator

In [155]:
from langchain_community.tools import tool
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage

# Model config
model = ChatGroq(model="llama-3.1-8b-instant")

# Tool creation
@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a * b

query = HumanMessage("What is 5 times 10 ?")
message_history = [query]

# Tool binding
model = model.bind_tools(tools=[multiply])

# Tool calling
model_tool_call = model.invoke(query.content)
message_history.append(model_tool_call)

# Tool execution
tool_result = multiply.invoke(model_tool_call.tool_calls[0])
message_history.append(tool_result)

# Sending back all the responses to the LLM
final_result = model.invoke(message_history)

# final output
print("\n========== Chat History ==========")
print(message_history)
print("\n========== Model Response ==========")
print(final_result)


========== Chat History ==========
[HumanMessage(content='What is 5 times 10 ?', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'rgw86j05k', 'function': {'arguments': '{"a":5,"b":10}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 220, 'total_tokens': 239, 'completion_time': 0.036654867, 'completion_tokens_details': None, 'prompt_time': 0.01299405, 'prompt_tokens_details': None, 'queue_time': 0.05290457, 'total_time': 0.049648917}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_4387d3edbb', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b841f-ab98-7f81-9894-6ca2fd692f58-0', tool_calls=[{'name': 'multiply', 'args': {'a': 5, 'b': 10}, 'id': 'rgw86j05k', 'type': 'tool_call'}], usage_metadata={'input_tokens': 220, 'output_tokens': 19, 'total_tokens': 239}), ToolMessa

##### Tool creating and binding using `StructuredTool` and `pydantic - BaseModel`

In [156]:
from langchain_community.tools import StructuredTool
from pydantic import BaseModel, Field
from langchain_groq import ChatGroq
from dotenv import load_dotenv
# Load api key
load_dotenv()

# Model config
model = ChatGroq(model="llama-3.1-8b-instant")

query = HumanMessage("What 10 times 20 ?")
message_history = [query]

# Input argument schema
class MultiplyTool(BaseModel):
    a: int = Field(required=True, description="First number to multiply")
    b: int = Field(required=True, description="Seconnd number to multiply")

# Create tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a * b

multiply_tool = StructuredTool.from_function(
    func=multiply,
    args_schema=MultiplyTool
)

# Bind tool
model = model.bind_tools(tools=[multiply_tool])

# Call tool
model_tool_call = model.invoke(query.content)
message_history.append(model_tool_call)

# Execute tool
tool_result = multiply_tool.invoke(model_tool_call.tool_calls[0])
message_history.append(tool_result)

# Send back tool result to the LLM
final_result = model.invoke(message_history)
print(final_result)

/var/folders/2n/vnp09xnn0s35m3xr_5c7_b300000gn/T/ipykernel_45723/3185241429.py:16: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  a: int = Field(required=True, description="First number to multiply")
/var/folders/2n/vnp09xnn0s35m3xr_5c7_b300000gn/T/ipykernel_45723/3185241429.py:17: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  b: int = Field(required=True, description="Seconnd number to multiply")


content='The result of 10 times 20 is 200.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 265, 'total_tokens': 278, 'completion_time': 0.025374116, 'completion_tokens_details': None, 'prompt_time': 0.040472421, 'prompt_tokens_details': None, 'queue_time': 0.056072699, 'total_time': 0.065846537}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_1151d4f23c', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'} id='lc_run--019b841f-ae08-74c0-850a-b3fe0099047c-0' usage_metadata={'input_tokens': 265, 'output_tokens': 13, 'total_tokens': 278}


##### Tool creating and binding using `BaseTool` class along with `pydantic model - BaseModel` and `typing - Type`

In [157]:
from langchain_community.tools import BaseTool
from typing import Type
from langchain_groq import ChatGroq
from pydantic import BaseModel, Field

# Input argument schema
class MultiplyToolSchema(BaseModel):
    a: int = Field(required=True, description="First number to multiply")
    b: int = Field(required=True, description="Second number to multiply")

# Tool class
class ToolForMultiply(BaseTool):
    name: str = "Multiply tool"
    description: str = "Product calculating tool"
    args_schema: Type[BaseModel] = MultiplyToolSchema

    def _run(self, a: int, b: int) -> int:
        """Multiply two numbers"""
        return a * b

# Tool binding
model = model.bind_tools(tools=[ToolForMultiply])

# Tool calling
tool_result = model.invoke(query.content)
message_history.append(tool_result)


/var/folders/2n/vnp09xnn0s35m3xr_5c7_b300000gn/T/ipykernel_45723/486439545.py:8: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  a: int = Field(required=True, description="First number to multiply")
/var/folders/2n/vnp09xnn0s35m3xr_5c7_b300000gn/T/ipykernel_45723/486439545.py:9: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  b: int = Field(required=True, description="Second number to multiply")
/Users/tusharshukla/GENAI/venv/lib/python3.11/site-packages/pydantic/json_schema.py:2448: PydanticJsonSchemaW

In [158]:
message_history

[HumanMessage(content='What 10 times 20 ?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'b3xtds9qs', 'function': {'arguments': '{"a":10,"b":20}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 235, 'total_tokens': 254, 'completion_time': 0.036154288, 'completion_tokens_details': None, 'prompt_time': 0.016787434, 'prompt_tokens_details': None, 'queue_time': 0.058873326, 'total_time': 0.052941722}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_4387d3edbb', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b841f-acf2-7a32-8674-208104f0afc5-0', tool_calls=[{'name': 'multiply', 'args': {'a': 10, 'b': 20}, 'id': 'b3xtds9qs', 'type': 'tool_call'}], usage_metadata={'input_tokens': 235, 'output_tokens': 19, 'total_tokens': 254}),
 ToolMessage(content='200', name='multiply